<a href="https://colab.research.google.com/github/yashthenuia/unsloth-finetune-ui/blob/main/colab/runner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install unsloth trl accelerate datasets bitsandbytes
print("Unsloth installed successfully!")

In [ ]:
!git clone https://github.com/yashthenuia/unsloth-finetune-ui.git
%cd unsloth-finetune-ui/colab


In [ ]:
from google.colab import files

uploaded = files.upload()
DATA_FILE = list(uploaded.keys())[0]
print("Uploaded:", DATA_FILE)


In [ ]:
import os

os.environ["MODEL_CHOICE"] = "unsloth/mistral-7b"
os.environ["EPOCHS"] = "2"
os.environ["BATCH_SIZE"] = "2"
os.environ["LR"] = "0.0002"
os.environ["DATA_FILE"] = DATA_FILE

!python3 trainer.py


Test your Trained model

In [ ]:
import os
import torch
import pandas as pd
from datasets import Dataset
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="customer_support_model",
    max_seq_length=2048,
    dtype = torch.float16,
    load_in_4bit=True,
)

# Enable inference mode
FastLanguageModel.for_inference(model)

def ask_question(question):
    """Generate response to a customer question."""
    if "mistral" in model_name.lower():
        prompt = f"<s>[INST] You are a helpful customer support agent. Answer clearly and professionally.\n\n{question} [/INST]"
    elif "llama" in model_name.lower():
        prompt = f"""system
You are a helpful customer support agent. Answer clearly and professionally.
user
{question}
assistant"""
    else:
        prompt = (
            "### Instruction:\nYou are a helpful customer support agent. "
            "Answer clearly and professionally.\n\n### Input:\n"
            f"{question}\n\n### Response:"
        )

    inputs = tokenizer([prompt], return_tensors="pt")
    if torch.cuda.is_available():
        inputs = {k: v.to("cuda") for k, v in inputs.items()}

    outputs = model.generate(
        **inputs,
        max_new_tokens=128,
        temperature=0.7,
        do_sample=True,
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract just the response text
    if "[/INST]" in response:
        response = response.split("[/INST]")[-1].strip()
    elif "assistant" in response:
        response = response.split("assistant")[-1].strip()
    elif "### Response:" in response:
        response = response.split("### Response:")[-1].strip()
    return response

In [ ]:
model_name = "unsloth/mistral-7b"
test_questions = [
    "How do I reset my password?",
    "My payment was declined",
    "How do I cancel my subscription?",
    "The app keeps crashing",
    "Where is my order?",
    "Do you offer student discounts?"
]

print("\n" + "="*60)
print("CUSTOMER SUPPORT AGENT TEST")
print("="*60)

for i, question in enumerate(test_questions, 1):
    print(f"\n Customer {i}: {question}")
    answer = ask_question(question)
    print(f"Support Agent: {answer}")
    print("-" * 60)

print("\nTesting complete!")